In [1]:
import sparql, re, types, itertools
from owlready2 import *

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


In [2]:
#Create new Ontology
onto = get_ontology("http://test.org/lmsontology.owl")

In [3]:
classes = ['Course', 'Lesson', 'LearningObject', 'DataElement', 'PParameter', 'User', 'Enrollment','LOProperty', 'UserPreference', 'UserCoursePreference']
#Add classes to ontology
for row in classes: 
    with onto:
        NewClass = types.new_class(row, (Thing,))

In [4]:
#Add properties to different classes
with onto:
    #Users
    class user_id (onto.User >> str): pass
    class name (onto.User >> str): pass
    class email (onto.User >> str): pass
    
    #User Preferences
    class user (onto.UserPreference >> str): pass
    class user_AR_FSLS (onto.UserPreference >> str): pass
    class user_AR_FSLSDegree (onto.UserPreference >> str): pass
    class user_AR_HM (onto.UserPreference >> str): pass
    class user_AR_HMDegree (onto.UserPreference >> str): pass
    class user_MediaPreference (onto.UserPreference >> str): pass
    class user_NavPreference (onto.UserPreference >> str): pass
    class user_SG_FSLS (onto.UserPreference >> str): pass
    class user_VV_FSLS (onto.UserPreference >> str): pass
    
    #User-Course Preferences
    class userInCourse (onto.UserCoursePreference >> str): pass
    class courseForUser (onto.UserCoursePreference >> str): pass
    class user_Language (onto.UserCoursePreference >> str): pass
    class user_LevelOfKnowledge (onto.UserCoursePreference >> str): pass
    class user_MotivationLevel (onto.UserCoursePreference >> str): pass
    class user_PedagogicalApp (onto.UserCoursePreference >> str): pass
    
    #Courses
    class course_id (onto.Course >> str): pass
    class course_title (onto.Course >> str): pass
    class publishedC (onto.Course >> str): pass
    
    #Learning objects
    class learning_object_id (onto.LearningObject >> str): pass
    class lo_title (onto.LearningObject >> str): pass
    class lo_concept (onto.LearningObject >> str): pass
    class lesson (onto.LearningObject >> str): pass
    
    #Learning Object properties
    class lo_id (onto.LOProperty >> str): pass
    class language (onto.LOProperty >> str): pass
    class interactivity_type (onto.LOProperty >> str): pass
    class lo_interactivity_level (onto.LOProperty >> str): pass
    class resource_type (onto.LOProperty >> str): pass
    class lo_format (onto.LOProperty >> str): pass
    class structure (onto.LOProperty >> str): pass
    class difficulty (onto.LOProperty >> str): pass
    class semantic_density (onto.LOProperty >> str): pass
    class purpose (onto.LOProperty >> str): pass
    
    #Lessons
    class lesson_id (onto.Lesson >> str): pass
    class lesson_title (onto.Lesson >> str): pass
    class course (onto.Lesson >> str): pass
    class publishedL (onto.Lesson >> str): pass
    
    #Personalisation Parameters
    class parameter (onto.PParameter >> str): pass
    class linguistic_term (onto.PParameter >> str): pass
    class degree (onto.PParameter >> str): pass
    class pp_de (onto.PParameter >> str): pass
    class iLevel (onto.PParameter >> str): pass
    class mValue (onto.PParameter >> str): pass
    class dElement (onto.PParameter >> str): pass
        
    #Enrollment Data Properties
    class enrollment_course_id (onto.Enrollment >> str): pass
    class enrollment_user_id (onto.Enrollment >> str): pass
              
    #Data Elements
    class metadata_value (onto.DataElement >> str): pass
    class element_nr (onto.DataElement >> str): pass
    class element_name (onto.DataElement >> str): pass
    class interactivity_level (onto.DataElement >> str): pass
    class de_pp (onto.DataElement >> str): pass

In [5]:
sql = 'http://localhost:2020/sparql'
rql = '<http://localhost:2020/resource/'
sur = sparql.unpack_row
sq = sparql.query
bReg = "\['|']"

In [6]:
#Add roles to Users
role_q = ('SELECT DISTINCT ?instance WHERE { ?instance a '+rql+'roles>} ORDER BY ?instance')

role_result = sq(sql, role_q)  
roles_dict = {}
for row in role_result:
    instance = re.sub('<IRI '+rql+'roles/id=|>|,|\(|\)','', str(row))
    name_q = ('SELECT ?name WHERE {'+rql+'roles/id='+instance+'> '+rql+'roles#name> ?name}')
    name_result =  sq(sql, name_q)
    
    #roles_dict = {}
    for nrow in name_result:
        roles_dict[instance] = str(nrow[0])
        #print (roles_dict)
        
        with onto:
            NewClass = types.new_class(str(nrow[0]), (onto.User,))

In [7]:
#Add instances and properties to User subclasses
user_q = ('SELECT DISTINCT ?instance WHERE { ?instance a '+rql+'users>} ORDER BY ?instance')

user_result = sq(sql, user_q)  
for row in user_result:
    user_dict={}
    instance = re.sub('<IRI '+rql+'users/id=|>|,|\(|\)','', str(row))
    email_q = ('SELECT ?email WHERE {'+rql+'users/id='+instance+'> '+rql+'users#email> ?email}')
    name_q = ('SELECT ?name WHERE {'+rql+'users/id='+instance+'> '+rql+'users#name> ?name}')
    type_q = ('SELECT ?type WHERE {'+rql+'users/id='+instance+'> '+rql+'users#type> ?type}')
    
    email_result =  sq(sql, email_q)
    name_result =  sq(sql, name_q)
    type_result =  sq(sql, type_q)
    
    for lrow, mrow, nrow in itertools.product(type_result, email_result, name_result):
        user_dict = {'id':instance, 'name':re.sub(bReg,'', str(sur(nrow))), 'email':re.sub(bReg,'', str(sur(mrow)))}
        for row in roles_dict:
            if (row == str(lrow[0])):
                class_str = str('onto.' + roles_dict[row])
                #print (user_dict)
                a = eval(class_str) (user_dict['name'], namespace = onto, user_id = [user_dict['id']], email = [user_dict['email']])

In [8]:
#Add instances and properties to User Preferences
userPref_q = ('SELECT DISTINCT ?instance WHERE { ?instance a '+rql+'user_preferences>} ORDER BY ?instance')

userPref_result = sq(sql, userPref_q)  
for row in userPref_result:
    userPref_dict={}
    instance = re.sub('<IRI '+rql+'user_preferences/id=|>|,|\(|\)','', str(row))
    
    userP_q = ('SELECT ?user_id WHERE {'+rql+'user_preferences/id='+instance+'> '+rql+'user_preferences#user_id> ?user_id}')
    user_AR_FSLS_q = ('SELECT ?ar_fsls WHERE {'+rql+'user_preferences/id='+instance+'> '+rql+'user_preferences#ar_fsls> ?ar_fsls}')
    user_AR_FSLSD_q = ('SELECT ?ar_fsls_d WHERE {'+rql+'user_preferences/id='+instance+'> '+rql+'user_preferences#ar_fsls_degree> ?ar_fsls_d}')
    user_AR_HM_q = ('SELECT ?ar_hm WHERE {'+rql+'user_preferences/id='+instance+'> '+rql+'user_preferences#ar_honey_mumford> ?ar_hm}')
    user_AR_HMD_q = ('SELECT ?ar_hm_d WHERE {'+rql+'user_preferences/id='+instance+'> '+rql+'user_preferences#ar_hm_degree> ?ar_hm_d}')
    user_MP_q = ('SELECT ?mp WHERE {'+rql+'user_preferences/id='+instance+'> '+rql+'user_preferences#media_preference> ?mp}')
    user_NP_q = ('SELECT ?np WHERE {'+rql+'user_preferences/id='+instance+'> '+rql+'user_preferences#nav_preference> ?np}')
    user_SG_FSLS_q = ('SELECT ?sg_fsls WHERE {'+rql+'user_preferences/id='+instance+'> '+rql+'user_preferences#sg_fsls> ?sg_fsls}')
    user_VV_FSLS_q = ('SELECT ?vv_fsls WHERE {'+rql+'user_preferences/id='+instance+'> '+rql+'user_preferences#vv_fsls> ?vv_fsls}')
    
    userP_result = sq(sql, userP_q)
    user_AR_FSLS_result = sq(sql, user_AR_FSLS_q)
    user_AR_FSLSD_result = sq(sql, user_AR_FSLSD_q)
    user_AR_HM_result = sq(sql, user_AR_HM_q)
    user_AR_HMD_result =sq(sql, user_AR_HMD_q)
    user_MP_result = sq(sql, user_MP_q)
    user_NP_result = sq(sql, user_NP_q)
    user_SG_FSLS_result = sq(sql, user_SG_FSLS_q)
    user_VV_FSLS_result = sq(sql, user_VV_FSLS_q)
    
    for arow, brow, crow, drow, erow, frow, grow, hrow, irow in itertools.product\
    (userP_result, user_AR_FSLS_result, user_AR_FSLSD_result, user_AR_HM_result, user_AR_HMD_result, user_MP_result, user_NP_result, user_SG_FSLS_result, user_VV_FSLS_result):
        userPref_dict = {'user':str(arow[0]),'user_AR_FSLS':re.sub(bReg,'', str(sur(brow))), 'user_AR_FSLSD':str(crow[0]),\
                         'user_AR_HM':re.sub(bReg,'', str(sur(drow))), 'user_AR_HMD':str(erow[0]),\
                         'user_MP':re.sub(bReg,'', str(sur(frow))), 'user_NP':re.sub(bReg,'', str(sur(grow))),\
                         'user_SG_FSLS':re.sub(bReg,'', str(sur(hrow))), 'user_VV_FSLS':re.sub(bReg,'', str(sur(irow)))}
        name = 'User'+userPref_dict['user']+ '_Preferences'
        onto.UserPreference (name, namespace = onto, user = [userPref_dict['user']],\
                            user_AR_FSLS = [userPref_dict['user_AR_FSLS']],\
                            user_AR_FSLSDegree = [userPref_dict['user_AR_FSLSD']],\
                            user_AR_HM = [userPref_dict['user_AR_HM']],\
                            user_AR_HMDegree = [userPref_dict['user_AR_HMD']],\
                            user_MediaPreference = [userPref_dict['user_MP']],\
                            user_NavPreference = [userPref_dict['user_NP']],\
                            user_SG_FSLS = [userPref_dict['user_SG_FSLS']],\
                            user_VV_FSLS = [userPref_dict['user_VV_FSLS']])

In [9]:
#Add instances and properties to User-Course Preferences
userCoursePref_q = ('SELECT DISTINCT ?instance WHERE { ?instance a '+rql+'user_course_values>} ORDER BY ?instance')

userCoursePref_result = sq(sql, userCoursePref_q)  
for row in userCoursePref_result:
    userCoursePref_dict={}
    instance = re.sub('<IRI '+rql+'user_course_values/id=|>|,|\(|\)','', str(row))
    
    userC_q = ('SELECT ?user_id WHERE {'+rql+'user_course_values/id='+instance+'> '+rql+'user_course_values#user_id> ?user_id}')
    courseU_q = ('SELECT ?course_id WHERE {'+rql+'user_course_values/id='+instance+'> '+rql+'user_course_values#course_id> ?course_id}')
    userL_q = ('SELECT ?language WHERE {'+rql+'user_course_values/id='+instance+'> '+rql+'user_course_values#language> ?language}')
    userLOK_q = ('SELECT ?lok  WHERE {'+rql+'user_course_values/id='+instance+'> '+rql+'user_course_values#level_of_knowledge> ?lok}')
    userML_q = ('SELECT ?ml WHERE {'+rql+'user_course_values/id='+instance+'> '+rql+'user_course_values#motivation_level> ?ml}')
    userPA_q = ('SELECT ?pa WHERE {'+rql+'user_course_values/id='+instance+'> '+rql+'user_course_values#pedagogical_app> ?pa}')
    
    userC_result = sq(sql, userC_q)
    courseU_result = sq(sql, courseU_q)
    userL_result = sq(sql, userL_q)
    userLOK_result = sq(sql, userLOK_q)
    userML_result = sq(sql, userML_q)
    userPA_result = sq(sql, userPA_q)
    
    for arow, brow, crow, drow, erow, frow in itertools.product\
    (userC_result, courseU_result, userL_result, userLOK_result, userML_result, userPA_result):
        userCoursePref_dict = {'user':str(arow[0]),'course':str(brow[0]),\
                         'user_L':re.sub(bReg,'', str(sur(crow))), 'user_LOK':re.sub(bReg,'', str(sur(drow))),\
                         'user_ML':re.sub(bReg,'', str(sur(erow))), 'user_PA':re.sub(bReg,'', str(sur(frow)))}
        name = 'User' + userCoursePref_dict['user'] + 'Course' + userCoursePref_dict['course'] + '_Preferences'
        onto.UserCoursePreference (name, namespace = onto, userInCourse = [userCoursePref_dict['user']],\
                            courseForUser = [userCoursePref_dict['course']],\
                            user_Language = [userCoursePref_dict['user_L']],\
                            user_LevelOfKnowledge = [userCoursePref_dict['user_LOK']],\
                            user_MotivationLevel = [userCoursePref_dict['user_L']],\
                            user_PedagogicalApp = [userCoursePref_dict['user_PA']])

In [10]:
#Add instances and properties to Courses
course_q = ('SELECT DISTINCT ?instance WHERE { ?instance a '+rql+'courses>} ORDER BY ?instance')

course_result = sq(sql, course_q)  
for row in course_result:
    course_dict = {}
    instance = re.sub('<IRI '+rql+'courses/id=|>|,|\(|\)','', str(row))
    title_q = ('SELECT ?course_title WHERE {'+rql+'courses/id='+instance+'> '+rql+'courses#course_title> ?course_title}')
    published_q = ('SELECT ?published WHERE {'+rql+'courses/id='+instance+'> '+rql+'courses#published> ?published}')
    
    title_result = sq(sql, title_q)
    published_result = sq(sql, published_q)
        
    for mrow, nrow in itertools.product(title_result, published_result):
        course_dict = {'id':instance, 'published':str(nrow[0]), 'course_title':re.sub(bReg,'', str(sur(mrow)))}
        #print (course_dict)
        onto.Course (course_dict['course_title'], namespace = onto, course_id = [course_dict['id']], publishedC = [course_dict['published']])

In [11]:
#Add instances to Enrollments
enrollment_q = ('SELECT DISTINCT ?instance WHERE { ?instance a '+rql+'enrollments>} ORDER BY ?instance')

enrollment_result = sq(sql, enrollment_q)  

enrollment_dict = {}

for row in enrollment_result:
    instance = re.sub('<IRI '+rql+'enrollments/id=|>|,|\(|\)','', str(row))
    user_id_q = ('SELECT ?user_id WHERE {'+rql+'enrollments/id='+instance+'> '+rql+'enrollments#user_id> ?user_id}')
    course_id_q = ('SELECT ?course_id WHERE {'+rql+'enrollments/id='+instance+'> '+rql+'enrollments#course_id> ?course_id}')
    
    user_result =  sq(sql, user_id_q)
    course_result =  sq(sql, course_id_q)
    
    for mrow, nrow in itertools.product (user_result, course_result):
        #enrollment_dict [instance] = {'user': str(mrow[0]), 'course':str(nrow[0])}
        #print (nrow[0])
        enrollment_dict = {'id':instance, 'enrollment_user_id':str(mrow[0]),'enrollment_course_id':str(nrow[0])}
        name = 'Enrollment_' + enrollment_dict['id']
        onto.Enrollment (name, namespace = onto, enrollment_course_id = [enrollment_dict['enrollment_course_id']], enrollment_user_id = [enrollment_dict['enrollment_user_id']])

In [12]:
#Add instances and properties to Lessons
lesson_q = ('SELECT DISTINCT ?instance WHERE { ?instance a '+rql+'lessons>} ORDER BY ?instance')

lesson_result = sq(sql, lesson_q)  
for row in lesson_result:
    lesson_dict = {}
    instance = re.sub('<IRI '+rql+'lessons/id=|>|,|\(|\)','', str(row))
    title_q = ('SELECT ?lesson_title WHERE {'+rql+'lessons/id='+instance+'> '+rql+'lessons#lesson_title> ?lesson_title}')
    published_q = ('SELECT ?published WHERE {'+rql+'lessons/id='+instance+'> '+rql+'lessons#published> ?published}')
    course_q = ('SELECT ?course_id WHERE {'+rql+'lessons/id='+instance+'> '+rql+'lessons#course_id> ?course_id}')
    
    title_result =  sq(sql, title_q)
    published_result =  sq(sql, published_q)
    course_result =  sq(sql, course_q)
    
    for lrow, mrow, nrow in itertools.product (title_result, published_result, course_result):
        lesson_dict = {'id':instance, 'lesson_title':re.sub(bReg,'', str(sur(lrow))), 'published':str(mrow[0]), 'course':str(nrow[0])}
        #print (lesson_dict)
        onto.Lesson (lesson_dict['lesson_title'], namespace = onto, lesson_id = [lesson_dict['id']], publishedL = [lesson_dict['published']], course = [lesson_dict['course']])

In [13]:
#Add instances and properties to Learning Objects
lo_q = ('SELECT DISTINCT ?instance WHERE { ?instance a '+rql+'learning_objects>} ORDER BY ?instance')

lo_result = sq(sql, lo_q)  
for row in lo_result:
    lo_dict = {}
    instance = re.sub('<IRI '+rql+'learning_objects/id=|>|,|\(|\)','', str(row))
    title_q = ('SELECT ?lo_title WHERE {'+rql+'learning_objects/id='+instance+'> '+rql+'learning_objects#lo_title> ?lo_title}')
    concept_q = ('SELECT ?concept WHERE {'+rql+'learning_objects/id='+instance+'> '+rql+'learning_objects#concept> ?concept}')
    lesson_q = ('SELECT ?lesson_id WHERE {'+rql+'learning_objects/id='+instance+'> '+rql+'learning_objects#lesson_id> ?lesson_id}')
    
    title_result =  sq(sql, title_q)
    concept_result =  sq(sql, concept_q)
    lesson_result =  sq(sql, lesson_q)
    
    for lrow, mrow, nrow in itertools.product (title_result, concept_result, lesson_result):
        lo_dict = {'id':instance, 'lo_title':re.sub(bReg,'', str(sur(lrow))), 'concept':re.sub(bReg,'', str(sur(mrow))), 'lesson':str(nrow[0])}
        #print (lo_dict)
        onto.LearningObject (lo_dict['lo_title'], namespace = onto, learning_object_id = [lo_dict['id']], lo_concept = [lo_dict['concept']], lesson = [lo_dict['lesson']])

In [14]:
#Add instances and properties to Learning Object Properties
loProperty_q = ('SELECT DISTINCT ?instance WHERE { ?instance a '+rql+'lo_properties>} ORDER BY ?instance')

loProperty_result = sq(sql, loProperty_q)  

for row in loProperty_result:
    lop_dict = {}
    instance = re.sub('<IRI '+rql+'lo_properties/id=|>|,|\(|\)','', str(row))
    lo_id_q = ('SELECT ?lo WHERE {'+rql+'lo_properties/id='+instance+'> '+rql+'lo_properties#lo> ?lo}')
    language_q = ('SELECT ?language WHERE {'+rql+'lo_properties/id='+instance+'> '+rql+'lo_properties#language> ?language}')
    interactivity_type_q = ('SELECT ?interactivity_type WHERE {'+rql+'lo_properties/id='+instance+'> '+rql+'lo_properties#interactivity_type> ?interactivity_type}')
    interactivity_level_q = ('SELECT ?interactivity_level WHERE {'+rql+'lo_properties/id='+instance+'> '+rql+'lo_properties#interactivity_level> ?interactivity_level}')
    resource_type_q = ('SELECT ?resource_type WHERE {'+rql+'lo_properties/id='+instance+'> '+rql+'lo_properties#resource_type> ?resource_type}')
    difficulty_q = ('SELECT ?difficulty WHERE {'+rql+'lo_properties/id='+instance+'> '+rql+'lo_properties#difficulty> ?difficulty}')
    format_q = ('SELECT ?format WHERE {'+rql+'lo_properties/id='+instance+'> '+rql+'lo_properties#format> ?format}')
    structure_q = ('SELECT ?structure WHERE {'+rql+'lo_properties/id='+instance+'> '+rql+'lo_properties#structure> ?structure}')
    semantic_density_q = ('SELECT ?semantic_density WHERE {'+rql+'lo_properties/id='+instance+'> '+rql+'lo_properties#semantic_density> ?semantic_density}')
    purpose_q = ('SELECT ?purpose WHERE {'+rql+'lo_properties/id='+instance+'> '+rql+'lo_properties#purpose> ?purpose}')
    
    lo_id_result =  sq(sql, lo_id_q)
    language_result =  sq(sql, language_q)
    interactivity_type_result =  sq(sql, interactivity_type_q)
    interactivity_level_result =  sq(sql, interactivity_level_q)
    resource_type_result =  sq(sql, resource_type_q)
    difficulty_result =  sq(sql, difficulty_q)
    format_result =  sq(sql, format_q)
    structure_result =  sq(sql, structure_q)
    semantic_density_result =  sq(sql, semantic_density_q)
    purpose_result =  sq(sql, purpose_q)
    
    for arow, brow, crow, drow, erow, frow, grow, hrow, irow, jrow in \
    itertools.product (lo_id_result, language_result, interactivity_type_result, interactivity_level_result, \
                       resource_type_result, difficulty_result, format_result, structure_result, semantic_density_result, purpose_result):
        lop_dict = {'id':instance, 'language':re.sub(bReg,'', str(sur(brow))),\
                    'interactivity_type':re.sub(bReg,'', str(sur(crow))), 'interactivity_level':re.sub(bReg,'', str(sur(drow))),\
                    'resource_type':re.sub(bReg,'', str(sur(erow))), 'difficulty':re.sub(bReg,'', str(sur(frow))),\
                    'lo_format':re.sub(bReg,'', str(sur(grow))), 'structure':re.sub(bReg,'', str(sur(hrow))),\
                    'semantic_density':re.sub(bReg,'', str(sur(irow))), 'purpose':re.sub(bReg,'', str(sur(jrow))), 'lo':str(arow[0])}
        name = 'LO_no_' + lop_dict['lo']
        onto.LOProperty (name, namespace = onto, language = [lop_dict['language']], interactivity_type = [lop_dict['interactivity_type']], lo_interactivity_level = [lop_dict['interactivity_level']],\
                        resource_type = [lop_dict['resource_type']], difficulty = [lop_dict['difficulty']], lo_format = [lop_dict['lo_format']], structure = [lop_dict['structure']],\
                        semantic_density = [lop_dict['semantic_density']], purpose = [lop_dict['purpose']], lo_id = [lop_dict['lo']])

In [15]:
#Add Personalisation Parameters values
pparameter_q = ('SELECT DISTINCT ?instance WHERE { ?instance a '+rql+'p_parameters>} ORDER BY ?instance')

pparameter_result = sq(sql, pparameter_q)  
parameter_dict = {}
for row in pparameter_result:
    instance = re.sub('<IRI '+rql+'p_parameters/id=|>|,|\(|\)','', str(row))
    parameter_q = ('SELECT ?parameter WHERE {'+rql+'p_parameters/id='+instance+'> '+rql+'p_parameters#parameter> ?parameter}')
        
    parameter_result =  sq(sql, parameter_q)
        
    for mrow in parameter_result:
        parameter_dict[instance] = str(mrow[0])
        with onto:
            NewClass = types.new_class(str(mrow[0]), (onto.PParameter,))

In [16]:
#Add Data Elements values
delement_q = ('SELECT DISTINCT ?instance WHERE { ?instance a '+rql+'data_elements>} ORDER BY ?instance')

delement_result = sq(sql, delement_q)  
delement_dict = {}
for row in delement_result:
    #delement_dict = {}
    instance = re.sub('<IRI '+rql+'data_elements/id=|>|,|\(|\)','', str(row))
    element_name_q = ('SELECT ?element_name WHERE {'+rql+'data_elements/id='+instance+'> '+rql+'data_elements#element_name> ?element_name}')
    element_nr_q = ('SELECT ?element_nr WHERE {'+rql+'data_elements/id='+instance+'> '+rql+'data_elements#element_nr> ?element_nr}')
    
    element_name_result =  sq(sql, element_name_q)
    element_nr_result =  sq(sql, element_nr_q)
    
    for mrow, nrow in itertools.product (element_name_result, element_nr_result):
        delement_dict [instance] = [str(nrow[0]), str(mrow[0])]
        with onto:
            NewClass = types.new_class(str(mrow[0]), (onto.DataElement,))

In [17]:
#Get data property values for Personalisation Parameters
pparameterV = ('SELECT DISTINCT ?instance WHERE { ?instance a '+rql+'delement_pparameters>} ORDER BY ?instance')

pparameterV_result = sq(sql, pparameterV)  

pparameterV_dict = {}

for row in pparameterV_result:
    parameterV_dict = {}
    instance = re.sub('<IRI '+rql+'delement_pparameters/id=|>|,|\(|\)','', str(row))
    
    parameterV_q = ('SELECT ?parameter WHERE {'+rql+'delement_pparameters/id='+instance+'> '+rql+'delement_pparameters#parameter> ?parameter}')
    data_elementV_q = ('SELECT ?data_element WHERE {'+rql+'delement_pparameters/id='+instance+'> '+rql+'delement_pparameters#data_element> ?data_element}')
    lingusticTerm_q = ('SELECT ?linguistic_term WHERE {'+rql+'delement_pparameters/id='+instance+'> '+rql+'delement_pparameters#linguistic_term> ?linguistic_term}')
    degree_q = ('SELECT ?degree WHERE {'+rql+'delement_pparameters/id='+instance+'> '+rql+'delement_pparameters#degree> ?degree}')
    mvalue_q = ('SELECT ?mvalue WHERE {'+rql+'delement_pparameters/id='+instance+'> '+rql+'delement_pparameters#metadata_value> ?mvalue}')
    ilevel_q = ('SELECT ?ilevel WHERE {'+rql+'delement_pparameters/id='+instance+'> '+rql+'delement_pparameters#interactivity_level> ?ilevel}')
        
    parameterV_result =  sq(sql, parameterV_q)
    data_elementV_result =  sq(sql, data_elementV_q)
    lingusticTerm_result =  sq(sql, lingusticTerm_q)
    mvalue_result =  sq(sql, mvalue_q)
    degree_result =  sq(sql, degree_q)
    ilevel_result =  sq(sql, ilevel_q)
    
    for irow, jrow, krow, lrow, mrow, nrow in itertools.product (data_elementV_result, mvalue_result, ilevel_result, parameterV_result, lingusticTerm_result, degree_result):
        parameterV_dict = {'data_element':str(irow[0]), 'parameter':str(lrow[0]), 'linguistic_term':re.sub(bReg,'', str(sur(mrow))), 'degree':str(nrow[0]),\
                           'mvalue':re.sub(bReg,'', str(sur(jrow))), 'ilevel':re.sub(bReg,'', str(sur(krow)))}
        #print (parameterV_dict)
        
        for drow, prow in itertools.product(delement_dict, parameter_dict):
            if ((prow == str(lrow[0])) & (drow == str(irow[0]))):
            #if (prow == str(lrow[0])):
                name = parameterV_dict['linguistic_term'] + '_' + parameterV_dict['degree'] + '_' + parameterV_dict['mvalue']
                class_str = str('onto.' + parameter_dict[prow])
                a = eval(class_str) (name, namespace = onto, linguistic_term = [parameterV_dict['linguistic_term']], parameter = [parameter_dict[prow]], dElement = [delement_dict[drow][1]],\
                                     mValue = [parameterV_dict['mvalue']], iLevel = [parameterV_dict['ilevel']], degree = [parameterV_dict['degree']], pp_de = [instance])
                
        for drow in delement_dict:
            if (drow == str(irow[0])):
                name = parameterV_dict['mvalue'] + '_' + parameterV_dict['ilevel'] + '_' + parameterV_dict['data_element']
                class_str = str('onto.' + delement_dict[drow][1])
                a = eval(class_str) (name, namespace = onto, element_name = [delement_dict[drow][1]], element_nr = [delement_dict[drow][0]], metadata_value = [parameterV_dict['mvalue']],\
                                     interactivity_level = [parameterV_dict['ilevel']], de_pp = [instance])

In [18]:
#parameterV_dict

In [19]:
#Object Properties    
with onto:
    class belongsToCourse (onto.Lesson >> onto.Course): pass
    class belongsToLesson (onto.LearningObject >> onto.Lesson): pass
    class loBelongsToCourse (onto.LearningObject >> onto.Course): pass
    class isEnrolledIn (onto.User >> onto.Course): pass
    class hasPreference (onto.User >> onto.PParameter): pass
    class relatesTo (onto.PParameter >> onto.DataElement): pass
    #class hasProperties (onto.LearningObject >> onto.LOProperty): pass
    class hasAttributes (onto.LOProperty >> onto.PParameter): pass
    class isDescribedBy (onto.LearningObject >> onto.DataElement): pass
    class hasCoursePreference (onto.UserCoursePreference >> onto.PParameter): pass
    class isSuitedTo (onto.User >> onto.LearningObject): pass
    class isCourseSuitedTo (onto.UserCoursePreference >> onto.LearningObject): pass

In [20]:
#SWRL rules
with onto:
    rule = Imp() #Relationship between lessons and courses
    rule.set_as_rule("""Lesson(?l), Course(?c), course_id(?c, ?cid), course(?l, ?cid) -> belongsToCourse (?l, ?c) """)
    rule2 = Imp() #Relationship between Learning Objects and Lessons
    rule2.set_as_rule("""LearningObject(?lo), Lesson(?l), lesson_id(?l, ?lid), lesson(?lo, ?lid) -> belongsToLesson (?lo, ?l) """)
    rule3 = Imp() #Relationship between users and courses enrolled
    rule3.set_as_rule("""Enrollment(?e), User(?u), enrollment_user_id(?e, ?euid), user_id(?u, ?euid), Course(?c), enrollment_course_id(?e, ?ecid), course_id(?c, ?ecid) -> isEnrolledIn(?u, ?c)""")
    #rule4 = Imp()
    #rule4.set_as_rule("""LearningObject(?lo), LOProperty(?lop), learning_object_id(?lo, ?loid), lo_id(?lop, ?loid) -> hasProperties(?lo, ?lop)""")
    
    rule4 = Imp() #Relationship between Personalisation Parameter and Data Element values
    rule4.set_as_rule("""PParameter(?pp), DataElement(?de), iLevel(?pp, ?il), interactivity_level(?de, ?il), mValue(?pp, ?v), metadata_value(?de, ?v), dElement(?pp, ?en), element_name(?de, ?en) -> relatesTo(?pp, ?de)""")
    
    rule5 = Imp() #Relationship between Personalisation Parameter and Data Element values
    rule5.set_as_rule("""PParameter(?pp), DataElement(?de), pp_de(?pp, ?id), de_pp(?de, ?id) -> relatesTo(?pp, ?de)""")
    
    rule6 = Imp()
    rule6.set_as_rule("""LearningObject(?lo), Lesson(?l), Course(?c), belongsToLesson (?lo, ?l), belongsToCourse (?l, ?c)  -> loBelongsToCourse(?lo, ?c)""")
    
    rule7 = Imp() #Relationship between Learning Objects and Difficulty
    rule7.set_as_rule("""LearningObject(?lo), LOProperty(?lop), learning_object_id(?lo, ?loid), lo_id(?lop, ?loid), Difficulty(?dif), difficulty(?lop, ?d), metadata_value(?dif, ?d) -> isDescribedBy(?lo, ?dif)""")
    rule8 = Imp() #Relationship between Learning Objects and Format
    rule8.set_as_rule("""LearningObject(?lo), LOProperty(?lop), learning_object_id(?lo, ?loid), lo_id(?lop, ?loid), Format(?f), lo_format(?lop, ?lo_f), metadata_value(?f, ?lo_f) -> isDescribedBy(?lo, ?f)""")
    rule9 = Imp() #Relationship between Learning Objects and Structure
    rule9.set_as_rule("""LearningObject(?lo), LOProperty(?lop), learning_object_id(?lo, ?loid), lo_id(?lop, ?loid), Structure(?s), structure(?lop, ?lo_s), metadata_value(?s, ?lo_s) -> isDescribedBy(?lo, ?s)""")
    rule10 = Imp() #Relationship between Learning Objects and Interactivity type
    rule10.set_as_rule("""LearningObject(?lo), LOProperty(?lop), learning_object_id(?lo, ?loid), lo_id(?lop, ?loid), Interactivity_Type(?it), interactivity_type(?lop, ?lo_it), metadata_value(?it, ?lo_it), interactivity_level(?it, ?lo_il), lo_interactivity_level(?lop, ?lo_il) -> isDescribedBy(?lo, ?it)""")
    rule11 = Imp()
    rule11.set_as_rule("""LearningObject(?lo), LOProperty(?lop), learning_object_id(?lo, ?loid), lo_id(?lop, ?loid), Language(?l), language(?lop, ?lo_l), metadata_value(?l, ?lo_l) -> isDescribedBy(?lo, ?l)""")
    rule12 = Imp()
    rule12.set_as_rule("""LearningObject(?lo), LOProperty(?lop), learning_object_id(?lo, ?loid), lo_id(?lop, ?loid), Learning_Resource_Type(?lrt), resource_type(?lop, ?lo_rt), metadata_value(?lrt, ?lo_rt) -> isDescribedBy(?lo, ?lrt)""")
    rule13 = Imp()
    rule13.set_as_rule("""LearningObject(?lo), LOProperty(?lop), learning_object_id(?lo, ?loid), lo_id(?lop, ?loid), Purpose(?p), purpose(?lop, ?lo_p), metadata_value(?p, ?lo_p) -> isDescribedBy(?lo, ?p)""")
    rule14 = Imp()
    rule14.set_as_rule("""LearningObject(?lo), LOProperty(?lop), learning_object_id(?lo, ?loid), lo_id(?lop, ?loid), Semantic_Density(?sd), semantic_density(?lop, ?lo_sd), metadata_value(?sd, ?lo_sd) -> isDescribedBy(?lo, ?sd)""")
    rule15 = Imp()
    rule15.set_as_rule("""User(?u), UserPreference(?up), user_id(?u, ?uid), user(?up, ?uid), Media_Preference(?mp), user_MediaPreference(?up, ?ump), linguistic_term(?mp, ?ump) -> hasPreference(?u, ?mp)""")
    rule16 = Imp()
    rule16.set_as_rule("""User(?u), UserPreference(?up), user_id(?u, ?uid), user(?up, ?uid), Navigation_Preference(?np), user_NavPreference(?up, ?unp), linguistic_term(?np, ?unp) -> hasPreference(?u, ?np)""")
    rule17 = Imp()
    rule17.set_as_rule("""User(?u), UserPreference(?up), user_id(?u, ?uid), user(?up, ?uid), Felder_Silverman_LS(?fsls), user_SG_FSLS(?up, ?usgfsls), linguistic_term(?fsls, ?usgfsls) -> hasPreference(?u, ?fsls)""")
    rule18 = Imp()
    rule18.set_as_rule("""User(?u), UserPreference(?up), user_id(?u, ?uid), user(?up, ?uid), Felder_Silverman_LS(?fsls), user_VV_FSLS(?up, ?uvvfsls), linguistic_term(?fsls, ?uvvfsls) -> hasPreference(?u, ?fsls)""")
    rule19 = Imp()
    rule19.set_as_rule("""User(?u), UserPreference(?up), user_id(?u, ?uid), user(?up, ?uid), Felder_Silverman_LS(?fsls), user_AR_FSLS(?up, ?uarfsls), linguistic_term(?fsls, ?uarfsls), user_AR_FSLSDegree(?up, ?uarfslsd), degree(?fsls, ?uarfslsd) -> hasPreference(?u, ?fsls)""")
    rule20 = Imp()
    rule20.set_as_rule("""User(?u), UserPreference(?up), user_id(?u, ?uid), user(?up, ?uid), Felder_Silverman_LS(?fsls), user_AR_HM(?up, ?uarhm), linguistic_term(?fsls, ?uarhm), user_AR_HMDegree(?up, ?uarhmd), degree(?fsls, ?uarhmd) -> hasPreference(?u, ?fsls)""")
    rule21 = Imp()
    rule21.set_as_rule("""UserCoursePreference(?ucp), Language_Preference(?lp), user_Language(?ucp, ?ul), linguistic_term(?lp, ?ul) -> hasCoursePreference(?ucp, ?lp)""")
    rule22 = Imp()
    rule22.set_as_rule("""UserCoursePreference(?ucp), Level_of_knowledge(?lok), user_LevelOfKnowledge(?ucp, ?ulok), linguistic_term(?lok, ?ulok) -> hasCoursePreference(?ucp, ?lok)""")
    rule23 = Imp()
    rule23.set_as_rule("""UserCoursePreference(?ucp), Motivation_Level(?ml), user_MotivationLevel(?ucp, ?uml), linguistic_term(?ml, ?uml) -> hasCoursePreference(?ucp, ?ml)""")
    rule24 = Imp()
    rule24.set_as_rule("""UserCoursePreference(?ucp), Pedagogical_Approach(?pa), user_PedagogicalApp(?ucp, ?upa), linguistic_term(?pa, ?upa) -> hasCoursePreference(?ucp, ?pa)""")
    
    rule25 = Imp()
    rule25.set_as_rule("""User(?u), LearningObject(?lo), hasPreference(?u, ?up), isDescribedBy(?lo, ?lom), relatesTo(?up, ?lom) -> isSuitedTo(?u, ?lo)""")

    
    rule26 = Imp()
    rule26.set_as_rule("""UserCoursePreference(?ucp), LearningObject(?lo), hasCoursePreference(?ucp, ?up), isDescribedBy(?lo, ?lom), relatesTo(?up, ?lom), courseForUser(?ucp, ?c), loBelongsToCourse(?lo, ?c) -> isCourseSuitedTo(?ucp, ?lo)""")

    
    
    
    

In [21]:
with onto:
    sync_reasoner_pellet(infer_property_values = True, infer_data_property_values = True)

* Owlready2 * Running Pellet...
    java -Xmx2000M -cp /home/ufuoma/.local/lib/python3.6/site-packages/owlready2/pellet/jcl-over-slf4j-1.6.4.jar:/home/ufuoma/.local/lib/python3.6/site-packages/owlready2/pellet/jgrapht-jdk1.5.jar:/home/ufuoma/.local/lib/python3.6/site-packages/owlready2/pellet/antlr-runtime-3.2.jar:/home/ufuoma/.local/lib/python3.6/site-packages/owlready2/pellet/commons-codec-1.6.jar:/home/ufuoma/.local/lib/python3.6/site-packages/owlready2/pellet/aterm-java-1.6.jar:/home/ufuoma/.local/lib/python3.6/site-packages/owlready2/pellet/jena-arq-2.10.0.jar:/home/ufuoma/.local/lib/python3.6/site-packages/owlready2/pellet/jena-core-2.10.0.jar:/home/ufuoma/.local/lib/python3.6/site-packages/owlready2/pellet/jena-tdb-0.10.0.jar:/home/ufuoma/.local/lib/python3.6/site-packages/owlready2/pellet/antlr-3.2.jar:/home/ufuoma/.local/lib/python3.6/site-packages/owlready2/pellet/xercesImpl-2.10.0.jar:/home/ufuoma/.local/lib/python3.6/site-packages/owlready2/pellet/httpclient-4.2.3.jar:/home

* Owlready * Adding relation lmsontology.simulation_100_application relatesTo lmsontology.application__1
* Owlready * Adding relation lmsontology.Humam hasPreference lmsontology.sound_100_audio
* Owlready * Adding relation lmsontology.Humam hasPreference lmsontology.sequential_100_linear
* Owlready * Adding relation lmsontology.Humam hasPreference lmsontology.verbal_100_lecture
* Owlready * Adding relation lmsontology.Humam hasPreference lmsontology.depth-first_100_hierarchical
* Owlready * Adding relation lmsontology.Humam hasPreference lmsontology.reflective_80_expositive
* Owlready * Adding relation lmsontology.Humam isSuitedTo lmsontology.LO28
* Owlready * Adding relation lmsontology.Humam isSuitedTo lmsontology.LO19
* Owlready * Adding relation lmsontology.Humam isSuitedTo lmsontology.LO21
* Owlready * Adding relation lmsontology.Humam isSuitedTo lmsontology.LO12
* Owlready * Adding relation lmsontology.Humam isSuitedTo lmsontology.LO16
* Owlready * Adding relation lmsontology.Hum

* Owlready * (NB: only changes on entities loaded in Python are shown, other changes are done but not listed)


In [22]:
onto.save(file = "draft_ontology", format = "rdfxml")

In [23]:
#with onto:
#    rule25 = Imp()
 #   rule25.set_as_rule("""User(?u), LearningObject(?lo), hasPreference(?u, ?up), isDescribedBy(?lo, ?up) -> isSuitableFor(?u, ?lo)""")

In [24]:
#with onto:
#    sync_reasoner_pellet(infer_property_values = True, infer_data_property_values = True)

In [27]:
print(onto.Student103.isSuitedTo)

[lmsontology.LO28, lmsontology.LO19, lmsontology.LO21, lmsontology.LO16, lmsontology.LO12, lmsontology.LO27, lmsontology.LO11, lmsontology.LO14, lmsontology.LO1, lmsontology.LO32, lmsontology.LO13, lmsontology.LO5, lmsontology.LO6, lmsontology.LO24, lmsontology.LO15, lmsontology.LO17, lmsontology.LO4, lmsontology.LO8, lmsontology.LO9, lmsontology.LO23, lmsontology.LO7, lmsontology.LO10, lmsontology.LO22, lmsontology.LO20, lmsontology.LO25, lmsontology.LO26, lmsontology.LO31, lmsontology.LO2, lmsontology.LO29, lmsontology.LO33, lmsontology.LO18, lmsontology.LO30]
